<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Module_2_HydroSAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code below is adapted from jhkennedy on Github. The script can be found [here](https://github.com/HydroSAR/HydroSAR/blob/develop/notebooks/hydrosar.ipynb)

In this Notebook we will obtain radiometrically-terrain-corrected (RTC) Sentinel-1 synthetic aperture radar (SAR) images from the Alaska Satellite Facility Distributed Active Archive Center (DAAC). We will then run the HydroSAR algorithm on these images to obtain binary water maps.

# Step 1: Install Python Packages

In [ ]:
!pip install boto3 hydrosar asf_tools hyp3_sdk

In [ ]:
#!pip install hydrosar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 647.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.5/91.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.9 MB/s eta 0:00:00
  Created wheel for pysheds: filename=pysheds-0.4-py3-none-any.whl size=90687 sha256=0e509504181e163a2dfbb7b5df4bafff84a0bd0911242530a3a4894da909c40d
  Stored in directory: /root/.cache/pip/wheels/8f/2b/0d/988f11a32332312c4d5dfe1720f891789e5b01853f6a842be6
Successfully built pysheds


In [ ]:
#!pip install asf_tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
#!pip install hyp3_sdk

# Step 2: Log in to Hyp3

While we could download Sentinel-1 RTC images from the ASF Vertex website, we can also request these images via a Python API called HYp3. HyP3 stands for the Hybrid Pluggable Processing Pipeline

In [ ]:
import hyp3_sdk as sdk
from google.colab import drive
import shutil
import asf_tools
from hydrosar.water_map import make_water_map

hyp3 = sdk.HyP3(prompt=True)

NASA Earthdata Login username: maynard_maganini
NASA Earthdata Login password: ··········


# Mount Google Drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/flood_mapping_intercomparison/hydrosar

/content/drive/MyDrive/flood_mapping_intercomparison/hydrosar


# Step 2: Get ASF Granules

Go to ASF and download. Follow the instructions [here](https://storymaps.arcgis.com/stories/2ead3222d2294d1fae1d11d3f98d7c35)

In [ ]:
granule1 = 'S1A_IW_GRDH_1SDV_20230616T010907_20230616T010932_049004_05E498_9B24'
granule2 = 'S1A_IW_GRDH_1SDV_20230616T010842_20230616T010907_049004_05E498_57EA'
granule3 = 'S1A_IW_GRDH_1SDV_20230616T010817_20230616T010842_049004_05E498_018B'
granule4 = 'S1A_IW_GRDH_1SDV_20230616T010817_20230616T010842_049004_05E498_018B'
granule5 = 'S1A_IW_GRDH_1SDV_20230616T010727_20230616T010752_049004_05E498_8BA8'

granules = [granule1, granule2, granule3, granule4, granule5]

In [ ]:
job_alpha = hyp3.submit_rtc_job(
    granule1, name='water-extent-example',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)

In [ ]:
job_alpha = hyp3.watch(job_alpha)

  0%|          | 0/1 [timeout in 10800 s]

In [ ]:
pwd

'/content/drive/MyDrive/flood_mapping_intercomparison/hydrosar'

In [ ]:
product_zip = job_alpha.download_files()[0]
shutil.unpack_archive(product_zip)
vv_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VV.tif'
vh_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VH.tif'
print(f'VV raster: {vv_raster}')
print(f'VH raster: {vv_raster}')

  0%|          | 0/1 [00:00<?, ?it/s]

S1A_IW_20230616T010907_DVP_RTC30_G_gpufed_0512.zip:   0%|          | 0/605775194 [00:00<?, ?it/s]

VV raster: S1A_IW_20230616T010907_DVP_RTC30_G_gpufed_0512/S1A_IW_20230616T010907_DVP_RTC30_G_gpufed_0512_VV.tif
VH raster: S1A_IW_20230616T010907_DVP_RTC30_G_gpufed_0512/S1A_IW_20230616T010907_DVP_RTC30_G_gpufed_0512_VV.tif


In [ ]:
#import asf_tools

In [ ]:
#from hydrosar.water_map import make_water_map

water_extent_raster_alpha = 'water_extent_alpha.tif'
make_water_map(water_extent_raster_alpha, vv_raster, vh_raster, tile_shape=(100, 100),
               max_vv_threshold=-15.5, max_vh_threshold=-23., hand_threshold=15., hand_fraction=0.8)

In [ ]:
ls

S1A_IW_20230616T010907_DVP_RTC30_G_gpufed_0512/     water_extent_alpha_VH_fuzzy.tif
S1A_IW_20230616T010907_DVP_RTC30_G_gpufed_0512.zip  water_extent_alpha_VH_initial.tif
water_extent_alpha_HAND.tif                         water_extent_alpha_VV_fuzzy.tif
water_extent_alpha.tif                              water_extent_alpha_VV_initial.tif


# Beta

In [ ]:
job_beta = hyp3.submit_rtc_job(
    granule2, name='water-extent-beta',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)

In [ ]:
job_beta = hyp3.watch(job_beta)

  0%|          | 0/1 [timeout in 10800 s]

In [ ]:
product_zip_beta = job_beta.download_files()[0]
shutil.unpack_archive(product_zip_beta)
vv_raster_beta = product_zip_beta.parent / product_zip_beta.stem / f'{product_zip_beta.stem}_VV.tif'
vh_raster_beta = product_zip_beta.parent / product_zip_beta.stem / f'{product_zip_beta.stem}_VH.tif'
print(f'VV raster: {vv_raster_beta}')
print(f'VH raster: {vv_raster_beta}')

  0%|          | 0/1 [00:00<?, ?it/s]

S1A_IW_20230616T010842_DVP_RTC30_G_gpufed_A2DB.zip:   0%|          | 0/609456284 [00:00<?, ?it/s]

VV raster: S1A_IW_20230616T010842_DVP_RTC30_G_gpufed_A2DB/S1A_IW_20230616T010842_DVP_RTC30_G_gpufed_A2DB_VV.tif
VH raster: S1A_IW_20230616T010842_DVP_RTC30_G_gpufed_A2DB/S1A_IW_20230616T010842_DVP_RTC30_G_gpufed_A2DB_VV.tif


In [ ]:
water_extent_raster_beta= 'water_extent_beta.tif'
make_water_map(water_extent_raster_beta, vv_raster_beta, vh_raster_beta, tile_shape=(100, 100),
               max_vv_threshold=-15.5, max_vh_threshold=-23., hand_threshold=15., hand_fraction=0.8)

# GAMMA

In [ ]:
job_gamma = hyp3.submit_rtc_job(
    granule3, name='water-extent-gamma',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)

In [ ]:
job_gamma = hyp3.watch(job_gamma)

  0%|          | 0/1 [timeout in 10800 s]

In [ ]:
product_zip_gamma = job_gamma.download_files()[0]
shutil.unpack_archive(product_zip_gamma)
vv_raster_gamma = product_zip_gamma.parent / product_zip_gamma.stem / f'{product_zip_gamma.stem}_VV.tif'
vh_raster_gamma = product_zip_gamma.parent / product_zip_gamma.stem / f'{product_zip_gamma.stem}_VH.tif'
print(f'VV raster: {vv_raster_gamma}')
print(f'VH raster: {vv_raster_gamma}')

  0%|          | 0/1 [00:00<?, ?it/s]

S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_CF4B.zip:   0%|          | 0/607198840 [00:00<?, ?it/s]

VV raster: S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_CF4B/S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_CF4B_VV.tif
VH raster: S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_CF4B/S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_CF4B_VV.tif


In [ ]:
water_extent_raster_gamma= 'water_extent_gamma.tif'
make_water_map(water_extent_raster_gamma, vv_raster_gamma, vh_raster_gamma, tile_shape=(100, 100),
               max_vv_threshold=-15.5, max_vh_threshold=-23., hand_threshold=15., hand_fraction=0.8)

#DELTA

In [ ]:
job_pt4 = hyp3.submit_rtc_job(
    granule4, name='water-extent-pt4',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)

In [ ]:
job_pt4 = hyp3.watch(job_pt4)

  0%|          | 0/1 [timeout in 10800 s]

In [ ]:
product_zip_pt4 = job_pt4.download_files()[0]
shutil.unpack_archive(product_zip_pt4)
vv_raster_pt4 = product_zip_pt4.parent / product_zip_pt4.stem / f'{product_zip_pt4.stem}_VV.tif'
vh_raster_pt4 = product_zip_pt4.parent / product_zip_pt4.stem / f'{product_zip_pt4.stem}_VH.tif'
print(f'VV raster: {vv_raster_pt4}')
print(f'VH raster: {vv_raster_pt4}')

  0%|          | 0/1 [00:00<?, ?it/s]

S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_041B.zip:   0%|          | 0/607199059 [00:00<?, ?it/s]

VV raster: S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_041B/S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_041B_VV.tif
VH raster: S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_041B/S1A_IW_20230616T010817_DVP_RTC30_G_gpufed_041B_VV.tif


In [ ]:
water_extent_raster_pt4= 'water_extent_pt4.tif'
make_water_map(water_extent_raster_pt4, vv_raster_pt4, vh_raster_pt4, tile_shape=(100, 100),
               max_vv_threshold=-15.5, max_vh_threshold=-23., hand_threshold=15., hand_fraction=0.8)

# EPSILON

In [ ]:
job_pt5 = hyp3.submit_rtc_job(
    granule5, name='water-extent-pt5',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)

In [ ]:
job_pt5 = hyp3.watch(job_pt5)

  0%|          | 0/1 [timeout in 10800 s]

In [ ]:
product_zip_pt5 = job_pt5.download_files()[0]
shutil.unpack_archive(product_zip_pt5)
vv_raster_pt5 = product_zip_pt5.parent / product_zip_pt5.stem / f'{product_zip_pt5.stem}_VV.tif'
vh_raster_pt5 = product_zip_pt5.parent / product_zip_pt5.stem / f'{product_zip_pt5.stem}_VH.tif'
print(f'VV raster: {vv_raster_pt5}')
print(f'VH raster: {vv_raster_pt5}')

  0%|          | 0/1 [00:00<?, ?it/s]

S1A_IW_20230616T010727_DVP_RTC30_G_gpufed_B93B.zip:   0%|          | 0/689747343 [00:00<?, ?it/s]

VV raster: S1A_IW_20230616T010727_DVP_RTC30_G_gpufed_B93B/S1A_IW_20230616T010727_DVP_RTC30_G_gpufed_B93B_VV.tif
VH raster: S1A_IW_20230616T010727_DVP_RTC30_G_gpufed_B93B/S1A_IW_20230616T010727_DVP_RTC30_G_gpufed_B93B_VV.tif


In [ ]:
water_extent_raster_pt5= 'water_extent_pt5.tif'
make_water_map(water_extent_raster_pt5, vv_raster_pt5, vh_raster_pt5, tile_shape=(100, 100),
               max_vv_threshold=-15.5, max_vh_threshold=-23., hand_threshold=15., hand_fraction=0.8)

# Looped Example

In [ ]:
cd looptest

[Errno 2] No such file or directory: 'looptest'
/content/drive/MyDrive/flood_mapping_intercomparison/hydrosar/looptest


In [ ]:
pwd

'/content/drive/My Drive/flood_mapping_intercomparison/hydrosar/looptest'

In [ ]:
import time

In [ ]:
granule1 = 'S1A_IW_GRDH_1SDV_20230616T010907_20230616T010932_049004_05E498_9B24'
granule2 = 'S1A_IW_GRDH_1SDV_20230616T010842_20230616T010907_049004_05E498_57EA'
granule3 = 'S1A_IW_GRDH_1SDV_20230616T010817_20230616T010842_049004_05E498_018B'
granule4 = 'S1A_IW_GRDH_1SDV_20230616T010817_20230616T010842_049004_05E498_018B'
granule5 = 'S1A_IW_GRDH_1SDV_20230616T010727_20230616T010752_049004_05E498_8BA8'

granules = [granule1, granule2, granule3, granule4, granule5]
n_granules = len(granules) # number of granules
job_name_stem = 'water-extent-pt'
raster_name_stem = 'water_extent_pt'

for k in range(n_granules):
  k += 1
  job_name = job_name_stem + str(k)
  print(job_name)
  raster_name = raster_name_stem + str(k)
  print(raster_name)

  # submit job to HyP3
  job = hyp3.submit_rtc_job(
    granules[k], name= job_name,
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
  )

  time.sleep(3000)
  # Unzip Files
  product_zip = job.download_files()[0]
  shutil.unpack_archive(product_zip)
  vv_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VV.tif'
  vh_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VH.tif'
  print(f'VV raster: {vv_raster}')
  print(f'VH raster: {vv_raster}')

  # Make Water Map
  make_water_map(raster_name, vv_raster_pt4, vh_raster_pt4, tile_shape=(100, 100),
                 max_vv_threshold=-15.5, max_vh_threshold=-23., hand_threshold=15., hand_fraction=0.8)




water-extent-pt1
water_extent_pt1


  0%|          | 0/1 [00:00<?, ?it/s]

IndexError: list index out of range

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
ls

drive/                                              water_extent_alpha.tif
S1A_IW_20230616T010907_DVP_RTC30_G_gpufed_9323/     water_extent_alpha_VH_fuzzy.tif
S1A_IW_20230616T010907_DVP_RTC30_G_gpufed_9323.zip  water_extent_alpha_VH_initial.tif
sample_data/                                        water_extent_alpha_VV_fuzzy.tif
water_extent_alpha_HAND.tif                         water_extent_alpha_VV_initial.tif


In [ ]:
cd drive/

/content/drive


In [ ]:
ls

MyDrive/  Shareddrives/


In [ ]:
cd MyDrive

/content/drive/MyDrive


In [ ]:
cd flood_mapping_intercomparison/hydrosar

/content/drive/MyDrive/flood_mapping_intercomparison/hydrosar


# Looped example

In [ ]:
jobs = []
count = 1
for i in granules:
  variable = hyp3.submit_rtc_job(
      i, name = 'water-extent-example',
      radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
      include_dem=True, include_rgb=True,
      speckle_filter=True, dem_matching=False
  )
  jobs.append(variable)

In [ ]:
for j in jobs:
  hyp3.watch(j)

  0%|          | 0/1 [timeout in 10800 s]

  0%|          | 0/1 [timeout in 10800 s]

  0%|          | 0/1 [timeout in 10800 s]

  0%|          | 0/1 [timeout in 10800 s]

  0%|          | 0/1 [timeout in 10800 s]

In [ ]:
import shutil

for k in jobs:
  product_zip = k.download_files()[0]
  shutil.unpack_archive(product_zip)
  vv_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VV.tif'
  vh_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VH.tif'
  print(f'VV raster: {vv_raster}')
  print(f'VH raster: {vv_raster}')

In [ ]:
import shutil

product_zip = job.download_files()[0]
shutil.unpack_archive(product_zip)
vv_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VV.tif'
vh_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VH.tif'
print(f'VV raster: {vv_raster}')
print(f'VH raster: {vv_raster}')

In [ ]:
from hydrosar.water_map import make_water_map

In [ ]:
joba = hyp3.submit_rtc_job(
    granny1, name='water-extent-example',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)
print(joba)

1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.


In [ ]:
jobb = hyp3.submit_rtc_job(
    granny2, name='water-extent-example',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)
print(jobb)

1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.


In [ ]:
jobc = hyp3.submit_rtc_job(
    granny3, name='water-extent-example',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)
print(jobc)

1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.


In [ ]:
jobd = hyp3.submit_rtc_job(
    granny4, name='water-extent-example',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)
print(jobd)

1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.


In [ ]:
jobe = hyp3.submit_rtc_job(
    granny5, name='water-extent-example',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)
print(jobe)

1 HyP3 Jobs: 0 succeeded, 0 failed, 0 running, 1 pending.


In [ ]:
joba = hyp3.watch(joba)
jobb = hyp3.watch(jobb)
jobc = hyp3.watch(jobc)
jobd = hyp3.watch(jobd)
jobe = hyp3.watch(jobe)

  0%|          | 0/1 [timeout in 10800 s]

  0%|          | 0/1 [timeout in 10800 s]

  0%|          | 0/1 [timeout in 10800 s]

  0%|          | 0/1 [timeout in 10800 s]

  0%|          | 0/1 [timeout in 10800 s]

In [ ]:
import shutil

product_zip = job.download_files()[0]
shutil.unpack_archive(product_zip)
vv_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VV.tif'
vh_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VH.tif'
print(f'VV raster: {vv_raster}')
print(f'VH raster: {vv_raster}')

NameError: name 'job' is not defined